<a href="https://colab.research.google.com/github/vilsonrodrigues/DataScience/blob/master/Guia_DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem de Dados

Importando bibliotecas

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

Lendo a base de dados

In [0]:
table = pd.read_csv("https://github.com/vilsonrodrigues/DataScienceEduc/raw/master/lopSubmissoes.csv")
table.head(2)

,Unnamed: 0,ano,data,idAluno,idLista,idTurma,matricula,porcentagemAcerto,quantidade,questao,titulo
0,0,2017.2,2017-08-14 16:19:03.629000+00:00,5991cd63b3c8b42f3d5da402,598cd57a3bc48946977dcf67,598e15296d8650eb27d52e3d,2.017010e+10,0.0,5.0,59885f8d684f4a3f78163c00,Lista de Laboratório 1 - Expressões Aritmética...
1,1,2017.2,2017-08-14 16:19:43.260000+00:00,5991cd63b3c8b42f3d5da402,598cd57a3bc48946977dcf67,598e15296d8650eb27d52e3d,2.017010e+10,0.0,5.0,59885f8d684f4a3f78163c00,Lista de Laboratório 1 - Expressões Aritmética...


Apagando coluna que não me interessa

In [0]:
table = table.drop(columns=["Unnamed: 0"])

Substituindo os valores nulos NaN por 0

In [0]:
table.replace(np.nan, 0, inplace=True) #onde inplace=True quer dizer substitua em. Ou seja, seria pra table receber as alterações sem precisar colocar direto

Passando os valores de objeto para o tipo DateTime

In [0]:
table["data"] = pd.to_datetime(table["data"])
table["data"] = table["data"].dt.tz_localize(None) #codigo para resolver o problema do fuso horario

Pegando apenas os dias da tabela data

In [0]:
table["dia"] = table["data"].dt.day
table.head(1)

Usando o loc para separar por turno as submissões

In [0]:
table["turno"]=1
table.loc[table["data"].dt.hour <6, "turno"]=0
table.loc[(table["data"].dt.hour >12) & (table["data"].dt.hour <=18), "turno"]=2
table.loc[table["data"].dt.hour >18, "turno"]=3
table.head(2)

Ordenando valores pela data

In [0]:
table.sort_values(by="data")

Trabalhando com time delta

Criando uma variável que tem um intervalo de 7 dias

In [0]:
t = pd.Timedelta("7 days")

Vamos usar ela para fazer buscas entre intervalos de dias específicos

In [0]:
d = pd.to_datetime("2019/02/11")

Criando os intervalos de semanas a partir da data de início do semestre, a cada 7 dias, ao total, 3 semanas

In [0]:
d11 = d+t
d12 = d11+t
d13 = d12+t
d14 = d13+t  

Buscando as submissões entre as semanas

In [0]:
v11 = table.loc[(table["data"]>d11) & (table["data"]<=d12),:]
v12 = table.loc[(table["data"]>d12) & (table["data"]<=d13),:]
v13 = table.loc[(table["data"]>d13) & (table["data"]<=d14),:]

Contando a quantidade de submissões dos alunos por semana

In [0]:
s11 = v11.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
s11.rename(columns={"questao":"qsemana1"},inplace=True)
s12 = v12.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
s12.rename(columns={"questao":"qsemana2"},inplace=True)
s13 = v13.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
s13.rename(columns={"questao":"qsemana3"},inplace=True)

Criando um dataframe novo para amarzenar os resultados

In [0]:
prov1 = pd.DataFrame()
prov1["idAluno"]=0 #esse sera o atributo que vai relacionar as 2 tabelas para depois juntar
prov1["qsemana1"]=0
prov1["qsemana2"]=0
prov1["qsemana3"]=0

Concatenação das tabelas

In [0]:
prov1 = prov1.append(s11,sort=False,ignore_index=True)
prov1 = prov1.append(s12,sort=False,ignore_index=True)
prov1 = prov1.append(s13,sort=False,ignore_index=True)

In [0]:
prov1.head()

Como terão dados como NaN por causa das buscas, vamos substituir eles por 0

In [0]:
prov1.replace(np.nan, 0, inplace=True)

Agora vamos somar os dados para ficar apenas 1 linha pra cada aluno

In [0]:
prov1 = prov1.groupby(["idAluno"]).sum()

In [0]:
prov1.head()

Ler outro .CSV, que contém as notas dos alunos

In [0]:
nota=pd.read_csv("https://raw.githubusercontent.com/vilsonrodrigues/DataScienceEduc/master/lopNotaProvaSubmissoesLista.csv")
nota=nota.drop(columns=["Unnamed: 0"])

Juntando as tabelas

In [0]:
df = pd.merge(nota,prov1,on="idAluno")

In [0]:
df.head()

#Cluster

In [0]:
from sklearn.cluster import KMeans

Selecionando as tabelas que eu quero trabalhar

In [0]:
x = table.iloc[:,24:32]
x.head(1)

Escolhendo a quantidade de grupos e depois treinando

In [0]:
kmeans = KMeans(2)
kmeans.fit(x)

Treinando o Kmeans

In [0]:
identified_clusters = kmeans.fit_predict(x)
identified_clusters

Inserindo os dados na tabela

In [0]:
table_clusters = df.copy()
table_clusters["cluster"] = identified_clusters
table_clusters.head(2)

Exibindo os gráficos 

In [0]:
plt.scatter("ano","qsemana8",data=table_clusters,c=table_clusters["cluster"],cmap="rainbow")
plt.show()

# Brincando com Gráficos

Gráfico de distribuição hexadecimal

In [0]:
sns.jointplot("ano","subp1",data=df,kind="hex")

Gráfico histograma dividindo por ano

In [0]:
g = sns.FacetGrid(tips, col="ano", margin_titles=True)#separe por
g.map(plt.hist, "mediaProvas", color="husl",data=df)#os dados

Boxplot

In [0]:
x = sns.FacetGrid(tips,col="ano",size=5,aspect=0.4)
x = x.map(sns.boxplot,"mediaProvas",data=df,color="r")